In [1]:
import mlflow
import pandas as pd
from mlflow.models import infer_signature
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Training

In [ ]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 42,
}

In [ ]:
# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_pred = lr.predict(X_test)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

### Logging

In [ ]:
# Run on terminal before proceeding: mlflow server --host 127.0.0.1 --port 8080
# This starts a local MLflow Tracking Server, which we connect to for logging our data

In [ ]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [ ]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Tracking Quickstart")

In [ ]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

### Inference

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [ ]:
predictions = loaded_model.predict(X_test)

In [ ]:
iris_feature_names = datasets.load_iris().feature_names

In [ ]:
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

In [ ]:
result[:4]

### Automatic logging

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Run on terminal before proceeding: mlflow server --host 127.0.0.1 --port 8080
# This starts a local MLflow Tracking Server, which we connect to for logging our data

In [3]:
mlflow.set_tracking_uri('http://127.0.0.1:8080')

In [4]:
mlflow.set_experiment("MLflow Tracking with Autologging")

2025/03/02 18:02:15 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Tracking with Autologging' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/147147490760873818', creation_time=1740949335221, experiment_id='147147490760873818', last_update_time=1740949335221, lifecycle_stage='active', name='MLflow Tracking with Autologging', tags={}>

In [5]:
mlflow.autolog()

2025/03/02 18:02:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [6]:
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

In [7]:
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

# MLflow triggers logging automatically upon model fitting
rf.fit(X_train, y_train)

2025/03/02 18:02:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cf592561a9d241ccb5411961c3d0ed3b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run stylish-mouse-960 at: http://127.0.0.1:8080/#/experiments/147147490760873818/runs/cf592561a9d241ccb5411961c3d0ed3b
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/147147490760873818


RandomForestRegressor(max_depth=6, max_features=3)